***Important*** DO NOT CLEAR THE OUTPUT OF THIS NOTEBOOK AFTER EXECUTION!!!

In [ ]:
# if the following command generates an error, you probably didn't enable
# the cluster security option "Allow API access to all Google Cloud services"
# under Manage Security → Project Access when setting up the cluster
!gcloud dataproc clusters list --region us-central1

# Imports & Setup

In [ ]:
!pip install -q google-cloud-storage==1.43.0
!pip install -q graphframes

In [ ]:
import pyspark
import sys
from collections import Counter, OrderedDict, defaultdict
import itertools
from itertools import islice, count, groupby
import pandas as pd
import os
import re
from operator import itemgetter
import nltk
from nltk.stem.porter import *
from nltk.corpus import stopwords
from time import time
from pathlib import Path
import pickle
import pandas as pd
from google.cloud import storage
import struct
import hashlib
def _hash(s):
    return hashlib.blake2b(bytes(s, encoding='utf8'), digest_size=5).hexdigest()

nltk.download('stopwords')

In [ ]:
######CHANGE FOLDER NAME!!!!!!!!!!!!!!!!!!!!!!!!#########################################################
FOLDER_NAME = "merged_corpus_index_two_words"

In [ ]:
# if nothing prints here you forgot to include the initialization script when starting the cluster
!ls -l /usr/lib/spark/jars/graph*

In [ ]:
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf, SparkFiles
from pyspark.sql import SQLContext
from graphframes import *

In [ ]:
spark

In [ ]:
# Put your bucket name below and make sure you can access it without an error
bucket_name = 'idx316179928316366087idx'
client = storage.Client()
blobs = client.list_blobs(bucket_name)
for b in blobs:
    print(b.name)

***GCP setup is complete!*** If you got here without any errors you've earned 10 out of the 35 points of this part.

# Building an inverted index

Here, we read the entire corpus to an rdd, directly from Google Storage Bucket and use your code from Colab to construct an inverted index.

In [ ]:
full_path = "gs://wikidata_preprocessed/*"
parquetFile = spark.read.parquet(full_path)

#######################################only for anchor text###################################
def id_to_anchor_text(id_and_text):
  return [(id,text) for id, text in id_and_text]

def reduce_anchor_text(list_of_text):
  return " ".join(list_of_text)

####################COMMENT OUT WHEN CALCULATING ANCHOR TEXT INDEX ###########################################
pages_links = parquetFile.select("id", "anchor_text").rdd
x = pages_links.flatMap(lambda x : id_to_anchor_text(x[1]))


anchor_doc_text_pairs = x.groupByKey().mapValues(reduce_anchor_text).map(lambda x:(x[1],x[0]))
title_doc_text_pairs = parquetFile.select("title", "id").rdd
body_doc_text_pairs = parquetFile.select("text", "id").rdd

We will count the number of pages to make sure we are looking at the entire corpus. The number of pages should be more than 6M

In [ ]:
# Count number of wiki pages for each
# Count number of wiki pages for each
N_of_anchor = anchor_doc_text_pairs.count()
N_of_title =  title_doc_text_pairs.count()
N_of_body = body_doc_text_pairs.count()

BODY_AVGDL = 319.52423534118395
TITLE_AVGDL = 2.525706459849008
ANCHOR_TEXT_AVGDL = 79.8183118653278

####title####
OPTIMAL_K_FOR_TITLE = 0.3
OPTIMAL_B_FOR_TITLE = 1
OPTIMAL_WEIGHT_FOR_TITLE = 0.34
####body####
OPTIMAL_K_FOR_BODY = 2.7
OPTIMAL_B_FOR_BODY = 0.05
OPTIMAL_WEIGHT_FOR_BODY = 0.13
####anchor####
OPTIMAL_K_FOR_ANCHOR = 0.1
OPTIMAL_B_FOR_ANCHOR = 0.2
OPTIMAL_WEIGHT_FOR_ANCHOR = 0.53








Let's import the inverted index module. Note that you need to use the staff-provided version called `inverted_index_gcp.py`, which contains helper functions to writing and reading the posting files similar to the Colab version, but with writing done to a Google Cloud Storage bucket.

In [ ]:
# if nothing prints here you forgot to upload the file inverted_index_gcp.py to the home dir
%cd -q /home/dataproc
!ls merged_all_inexes_with_bm_25_score.py

In [ ]:
# adding our python module to the cluster
sc.addFile("/home/dataproc/merged_all_inexes_with_bm_25_score.py")
sys.path.insert(0,SparkFiles.getRootDirectory())

In [ ]:
from merged_all_inexes_with_bm_25_score import InvertedIndex


**YOUR TASK (10 POINTS)**: Use your implementation of `word_count`, `reduce_word_counts`, `calculate_df`, and `partition_postings_and_write` functions from Colab to build an inverted index for all of English Wikipedia in under 2 hours.

A few notes:
1. The number of corpus stopwords below is a bit bigger than the colab version since we are working on the whole corpus and not just on one file.
2. You need to slightly modify your implementation of  `partition_postings_and_write` because the signature of `InvertedIndex.write_a_posting_list` has changed and now includes an additional argument called `bucket_name` for the target bucket. See the module for more details.
3. You are not allowed to change any of the code not coming from Colab.

In [ ]:
from nltk.stem.porter import *

stemmer = PorterStemmer()


english_stopwords = frozenset(stopwords.words('english'))
corpus_stopwords = ["category", "references", "also", "external", "links",
                    "may", "first", "see", "history", "people", "one", "two",
                    "part", "thumb", "including", "second", "following",
                    "many", "however", "would", "became"]

all_stopwords = english_stopwords.union(corpus_stopwords)
RE_WORD = re.compile(r"""[\#\@\w](['\-]?\w){2,24}""", re.UNICODE)

NUM_BUCKETS = 124
def token2bucket_id(token):
  return int(_hash(token),16) % NUM_BUCKETS

# PLACE YOUR CODE HERE
def word_count(text, id):
  ''' Count the frequency of each word in `text` (tf) that is not included in
  `all_stopwords` and return entries that will go into our posting lists.
  Parameters:
  -----------
    text: str
      Text of one document
    id: int
      Document id
  Returns:
  --------
    List of tuples
      A list of (token, (doc_id, tf)) pairs
      for example: [("Anarchism", (12, 5)), ...]
  '''
  tokens = [token.group() for token in RE_WORD.finditer(text.lower())]
  #ADDED stopwords removal
  tokens = [stemmer.stem(token) for token in tokens if token not in all_stopwords]
  two_word_tokens = [tokens[i] + " " + tokens[i+1] for i in range(len(tokens)-1)]
  tokens = tokens + two_word_tokens
  # YOUR CODE HERE
  count = Counter(tokens)
  if len(count)==0:
      return []
  #### ADDED doc_len and max_tf to posting lists
  #return [(word,(id,tf,max_tf, doc_len)) for word,tf in count.items() if word not in all_stopwords]
  doc_len = len(tokens) #to normalize tf
  return [(word,(id,tf,doc_len)) for word,tf in count.items()]



def calculate_df(postings):
  ''' Takes a posting list RDD and calculate the df for each token.
  Parameters:
  -----------
    postings: RDD
      An RDD where each element is a (token, posting_list) pair.
  Returns:
  --------
    RDD
      An RDD where each element is a (token, df) pair.
  '''
  # YOUR CODE HERE
  return postings.mapValues(lambda x: len(x))


def word_posting2bucket_and_wpl(word, posting_list):
  return token2bucket_id(word),(word,posting_list)

def write_bucket_to_disk(bucket_id, list_of_words_postings_tuples):
  word2posting_locs = InvertedIndex().write_a_posting_list((bucket_id, list_of_words_postings_tuples),bucket_name, FOLDER_NAME)
  return word2posting_locs

def partition_postings_and_write(postings):
  ''' A function that partitions the posting lists into buckets, writes out
  all posting lists in a bucket to disk, and returns the posting locations for
  each bucket. Partitioning should be done through the use of `token2bucket`
  above. Writing to disk should use the function  `write_a_posting_list`, a
  static method implemented in inverted_index_colab.py under the InvertedIndex
  class.
  Parameters:
  -----------
    postings: RDD
      An RDD where each item is a (w, posting_list) pair.
  Returns:
  --------
    RDD
      An RDD where each item is a posting locations dictionary for a bucket. The
      posting locations maintain a list for each word of file locations and
      offsets its posting list was written to. See `write_a_posting_list` for
      more details.
  '''
  # YOUR CODE HERE
  bucket2posting_lists=postings.map(lambda x: word_posting2bucket_and_wpl(x[0],x[1])).groupByKey()#.mapValues().reduce_word_counts)
  x= bucket2posting_lists.map(lambda x: write_bucket_to_disk(x[0],x[1]))
  return x


In [ ]:
from math import log10
"""
numerator = (k + 1) * tf
denumerator = tf + k * (1 - b + ((b * doc_len) / AVGDL))
(numerator / denumerator) * idf[term]
"""
def calculate_index_bm_25(pls, k, b,AVGDL, weight, N):
    df = len(pls)
    idf = log10((N + 1)/df)
    posting_list_of_doc_id_score = []
    for doc_id, tf, doc_len in pls:
        numerator = (k + 1) * tf
        denumerator = tf + k * (1 - b + ((b * doc_len) / AVGDL))
        score = (numerator / denumerator) * idf
        weighted_score = score*weight
        posting_list_of_doc_id_score.append((doc_id, weighted_score))
    return posting_list_of_doc_id_score


In [ ]:
# title word counts map
title_word_counts = title_doc_text_pairs.flatMap(lambda x: word_count(x[0], x[1]))
title_postings = title_word_counts.groupByKey().mapValues(lambda x: calculate_index_bm_25(x,OPTIMAL_K_FOR_TITLE, OPTIMAL_B_FOR_TITLE, TITLE_AVGDL , OPTIMAL_WEIGHT_FOR_TITLE, N_of_title))
# body word counts map
body_word_counts = body_doc_text_pairs.flatMap(lambda x: word_count(x[0], x[1]))
body_postings = body_word_counts.groupByKey().filter(lambda x: len(x[1])>50).mapValues(lambda x: calculate_index_bm_25(x,OPTIMAL_K_FOR_BODY, OPTIMAL_B_FOR_BODY, BODY_AVGDL, OPTIMAL_WEIGHT_FOR_BODY, N_of_body)) ## body is filtered!
# anchor word counts map
anchor_word_counts = anchor_doc_text_pairs.flatMap(lambda x: word_count(x[0], x[1]))
anchor_postings = anchor_word_counts.groupByKey().mapValues(lambda x: calculate_index_bm_25(x, OPTIMAL_K_FOR_ANCHOR, OPTIMAL_B_FOR_ANCHOR, ANCHOR_TEXT_AVGDL, OPTIMAL_WEIGHT_FOR_ANCHOR, N_of_anchor))

In [ ]:
def group_all_doc_ids_toghether(list_of_tuples):
    count = Counter()
    for doc_id, score in list_of_tuples:
        count[doc_id] += score
    return list(count.items())


final_postings_after_merge = title_postings.union(body_postings).union(anchor_postings).reduceByKey(lambda a,b: a+b).mapValues(lambda x: group_all_doc_ids_toghether(x))

In [ ]:
w2df = calculate_df(final_postings_after_merge)
w2df_dict = w2df.collectAsMap()
# partition posting lists and write out
_ = partition_postings_and_write(final_postings_after_merge).collect()

In [ ]:
# collect all posting lists locations into one super-set
super_posting_locs = defaultdict(list)
for blob in client.list_blobs(bucket_name, prefix=FOLDER_NAME):
  if not blob.name.endswith("pickle"):
    continue
  with blob.open("rb") as f:
    posting_locs = pickle.load(f)
    for k, v in posting_locs.items():
      super_posting_locs[k].extend(v)

Putting it all together

In [ ]:
# Create inverted index instance
inverted = InvertedIndex()
# Adding the posting locations dictionary to the inverted index
inverted.posting_locs = super_posting_locs
# Add the token - df dictionary to the inverted index
inverted.df = w2df_dict
inverted.AVGDL = 0
# write the global stats out
inverted.write_index('.', 'index')
# upload to gs
index_src = "index.pkl"
index_dst = f'gs://{bucket_name}/{FOLDER_NAME}/{index_src}'
!gsutil cp $index_src $index_dst

In [ ]:
!gsutil ls -lh $index_dst



